In [4]:
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
import os
from helpers import *
from preprocessing import *

In [2]:
x_train, x_test, y_train, train_ids, test_ids=load_csv_data("data")

## Preprocessing and Scalings

In [5]:
DATA_FOLDER = 'data/'

In [6]:
names = np.genfromtxt(DATA_FOLDER + 'x_train.csv', delimiter=",", dtype=str, max_rows=1)
names = np.delete(names, 0)
names_map = {}
for i in range(len(names)):
    names_map[names[i]] = i

In [7]:
y_train = np.where(y_train == -1, 0, y_train)
x_tr = x_train.copy()
y_tr = y_train.copy()


In [8]:
#clean the data and return the cleaned dataset, the y values, the mean and median dictionnaries to use to clean other sets (test set for example)
x_tr_clean, y_tr_clean, mean_dico, median_dico,intersting_features = clean_data(names_map, x_tr, y_tr, is_y=True, is_train_data=True)

#if we don't have the y values, we use is_y=False.
x_test_clean, _, _, _, _ = clean_data(names_map, x_test, y_raw=None, is_y=False, is_train_data=False, mean_dico=mean_dico, median_dico=median_dico)

#compute the scaled train dataset and the mean and std used for the scaling
x_tr_scaled, train_mean, train_std = scale_data(x_tr_clean, names_map, is_train_data=True)

#scale the test set using the mean and std computed on the train set
x_test_scaled, _, _ = scale_data(x_test_clean, names_map, is_train_data=False, train_mean=train_mean, train_std=train_std)

In [9]:
x_tr_scaled.shape

(328135, 321)

## Feature selection

We chose some initial features by reading the report and assessing how they seemed relevant for our predictions.